In [1]:
import pandas as pd 
import numpy as np  

from sklearn.model_selection import train_test_split 
import tensorflow as tf 
from tensorflow import keras
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/Churn_Modelling_Cleaned.csv')
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,1,1,1,0.506735,1,False,False
1,0.516,0,0.310811,0.1,0.334031,1,0,1,0.562709,0,False,True
2,0.304,0,0.324324,0.8,0.636357,3,1,0,0.569654,1,False,False
3,0.698,0,0.283784,0.1,0.000000,2,0,0,0.469120,0,False,False
4,1.000,0,0.337838,0.2,0.500246,1,1,1,0.395400,0,False,True


In [3]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(20, input_shape=(11,), activation='relu'), # "more hidden layers can be added" 
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', 
              loss=loss, 
              metrics=['accuracy']) 
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)

    else: 
        model.fit(X_train, y_train, epochs=100, class_weights=weights)

    print("Model evaluation: \n", model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification report: \n", classification_report(y_test, y_preds))

    return y_preds

In [4]:
count_class_0, count_class_1 = df['Exited'].value_counts()

print('Count of class 0: ', count_class_0)
print('Count of class 1: ', count_class_1)

Count of class 0:  7963
Count of class 1:  2037


From above we can see that the customers who exited (class 0) has 7963 records whereas that of class 1 (didn't exit) is 2037. This is a significant difference creating biasness in the dataset.

In [5]:
# splitting dataframes with respect to classes 0 and 1
df_with_class_0 = df[df['Exited'] == 0] 
df_with_class_1 = df[df['Exited'] == 1]

### Applying "over sampling minority class by duplication" to tackle imbalaced classes in data

In [6]:
df_class_1_over = df_with_class_1.sample(count_class_0, replace=True) # df.sample randomly creates copy of the existing samples

df_with_over_sampling_minority_class = pd.concat([df_with_class_0, df_class_1_over], axis=0)
df_with_over_sampling_minority_class.shape

(15926, 12)

In [7]:
print('Random over-sampling: ')
print(df_with_over_sampling_minority_class['Exited'].value_counts())

Random over-sampling: 
Exited
0    7963
1    7963
Name: count, dtype: int64


Now we have equal number of classes for Exited=0 and Exited=1

In [8]:
X = df_with_over_sampling_minority_class.drop('Exited', axis='columns')
y = df_with_over_sampling_minority_class['Exited']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y) #"stratify=y" ensures that the class distribution in y_train and y_test is the same as in y

In [10]:
y_train.value_counts()

Exited
0    6370
1    6370
Name: count, dtype: int64

In [11]:
y_test.value_counts()

Exited
0    1593
1    1593
Name: count, dtype: int64

We can see from above two cells that in both datasets, the number classes for 0s and 1s are same

In [12]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5896 - loss: 0.6657
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6803 - loss: 0.5972
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6990 - loss: 0.5759
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7093 - loss: 0.5623
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7189 - loss: 0.5515
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7361 - loss: 0.5369
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7402 - loss: 0.5296
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7390 - loss: 0.5308
Epoch 9/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7524 - loss: 0.5194
Epoch 10/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7445 - loss: 0.5195
Epoch 11/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7538 - loss: 0.5110
Epoch 12/100
399/399 ━━━━━━━━━━━━━━━━━━━━

Balanced performance – The model performs well for both classes, with precision and recall similar for each class (class 0: 78%, 82%; class 1: 81%, 77%).

Class balance – The dataset is perfectly balanced, with 1593 instances of class 0 and 1593 instances of class 1, leading to consistent performance across both classes.

Precision-recall tradeoff – The model favors precision slightly for class 1 (81%) over recall (77%), while class 0 has a stronger recall (82%) than precision (78%). The F1-scores for both classes are similar (0.80 for class 0 and 0.79 for class 1).